In [1]:
from huggingface_hub import login
access_token = "secret-token"
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/idsia/.cache/huggingface/token
Login successful


In [2]:
import torch
import datasets
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, TFBertForQuestionAnswering,TFAutoModelWithLMHead

In [3]:
GPU_use = 0
st = "cuda:"+str(GPU_use)
torch.cuda.set_device(GPU_use)

In [4]:
ds = datasets.load_dataset('marcomaccarini/blind_3_7_tasks')

In [5]:
trn = ds['train']

In [6]:
base_model = 'meta-llama/Meta-Llama-3-8B'
tokr = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained("marcomaccarini/FLARE", torch_dtype=torch.bfloat16, device_map=GPU_use,token=access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
fmt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: {}
===
{}
ASSISTANT:"""

In [8]:
def sql_prompt(d): 
    return fmt.format(d["context"], d["question"])
def question(table, quest):
    tst = dict(**trn[8])
    tst['context'] = table
    tst['question'] = quest
    return sql_prompt(tst)

In [9]:
t = 'table([ eof x: 345 y: 279 z: 348, gripper: open , black-cup x: -154 y: 127 z: 180, white-cup x: -115 y: 359 z: 60, box x: -310 y: 134 z: 100, green-cylinder x: 125 y: -32 z: 80 or: 142, green-cube x: -190 y: -190 z: 80 or: 83, grey-cube x: 536 y: -222 z: 80 or: 96, red-cube x: -32 y: 58 z: 80 or: 157, yellow-ball x: -21 y: 30 z: 20 or: 41, banana x: 2 y: 53 z: 20 or: 9, remote x: -48 y: 31 z: 30 or: 69, pen x: -53 y: -59 z: 10 or: 174 ])'
q = 'pick green-cube and place to black-cup'

In [10]:
%%time
import time
start = time.time()
test = question(t,q)
toks = tokr(test, return_tensors="pt")
res = model.generate(**toks.to(st), max_new_tokens=100, top_p = 0).to('cpu')
done = time.time()
elapsed = done - start
print(elapsed)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2.026280164718628
CPU times: user 1.89 s, sys: 142 ms, total: 2.03 s
Wall time: 2.03 s


In [11]:
print(tokr.batch_decode(res)[0].replace("*","\n"))

<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: table([ eof x: 345 y: 279 z: 348, gripper: open, black-cup x: -154 y: 127 z: 180, white-cup x: -115 y: 359 z: 60, box x: -310 y: 134 z: 100, green-cylinder x: 125 y: -32 z: 80 or: 142, green-cube x: -190 y: -190 z: 80 or: 83, grey-cube x: 536 y: -222 z: 80 or: 96, red-cube x: -32 y: 58 z: 80 or: 157, yellow-ball x: -21 y: 30 z: 20 or: 41, banana x: 2 y: 53 z: 20 or: 9, remote x: -48 y: 31 z: 30 or: 69, pen x: -53 y: -59 z: 10 or: 174 ])
===
pick green-cube and place to black-cup
ASSISTANT: -190+0;-190+0;80+300;83,-190+0;-190+0;80+0;83,close,-190+0;-190+0;80+300;83,-154+0;127+0;180+300;0,-154+0;127+0;180+80;0,open,home<|end_of_text|>


In [12]:
coord = tokr.batch_decode(res)[0].split(":")[-1].replace(" ","").replace(")<|end_of_text|>","")

In [13]:
def parse_inference(xyz_theta):
    if '+' in xyz_theta: 
        splitted = xyz_theta.split("+")
        x = int(splitted[0])+int(splitted[1])
    elif '-' in xyz_theta:
        splitted = xyz_theta.split("-")
        x = int(splitted[0])-int(splitted[1])
    else:
        splitted = xyz_theta.split("-")
        x = int(splitted[0])
    return x

def clean_inference(xyz_theta):
    a = []
    for elements in xyz_theta:
        if elements != "":
            a.append(elements)
    return a

step = []
for c in coord.split(","): 
    xyz_theta = clean_inference(c.split(";"))
    #print(xyz_theta)
    if len(xyz_theta)==1:
        step.append(xyz_theta[0].replace("<|end_of_text|>",""))
    elif len(xyz_theta)==4: 
        x = parse_inference(xyz_theta[0])
        y = parse_inference(xyz_theta[1])
        z = parse_inference(xyz_theta[2])
        step.append([x,y,z, int(xyz_theta[3])])
    else:
        step.append([int(xyz_theta[0]),int(xyz_theta[1]), int(xyz_theta[2].replace("<|end_of_text|>","")), 0])



In [14]:
step

[[-190, -190, 380, 83],
 [-190, -190, 80, 83],
 'close',
 [-190, -190, 380, 83],
 [-154, 127, 480, 0],
 [-154, 127, 260, 0],
 'open',
 'home']